In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss,accuracy_score, confusion_matrix, classification_report, roc_auc_score, mean_squared_error, f1_score

In [9]:
train=pd.read_csv("/kaggle/input/mle-ese-mock/train (5).csv")
test=pd.read_csv("/kaggle/input/mle-ese-mock/test (4).csv")
train.info()

In [10]:
train.head()

Train shape: (2250, 16)
Test shape: (750, 15)


NameError: name 'sample_submission' is not defined

In [ ]:
test.head()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train_ids = train['id']    # remove only if id column present in the train dataset
train.drop(columns=['id'], inplace=True)

In [ ]:
X = train.drop(columns=['quality_grade'])   # put your target column here
y = train['quality_grade']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns


In [ ]:
#nums_cols=x.select_dtypes(include=["int64","float64"]).columns
X[numeric_features].hist(figsize=(14,10))
plt.suptitle("Numerical Feature Distribution")
plt.show()

In [ ]:
plt.figure(figsize=(8,10))
sns.heatmap(train[numeric_features].corr(),annot=False,cmap="coolwarm")
plt.title("Correlation between numerical features")
plt.show()

In [ ]:
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [ ]:
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])


In [ ]:
X_train

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier(
    loss="log_loss",        # IMPORTANT
    max_iter=100,
    learning_rate=0.05,
    max_depth=6,
    min_samples_leaf=30,
    l2_regularization=0.1,
    max_bins=255,
    random_state=42
)

In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
pipeline.fit(X_train, y_train)
y_pred_proba = pipeline.predict_proba(X_test)
loss = log_loss(y_test, y_pred_proba)
print(f"\n✅ Log Loss on validation set: {loss:.5f}")

In [ ]:
y_pred=pipeline.predict(X_test)
accuracy=accuracy_score(y_pred,y_test)
print(accuracy)

In [ ]:
cm=confusion_matrix(y_pred,y_test)
print(cm)

In [ ]:
f1=f1_score(y_test,y_pred,average='weighted')
print(f1)

In [ ]:
mean_square=mean_squared_error(y_test,y_pred,squared=False)
print(mean_square)

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(cm,annot=True,fmt="d",cmap="Blues")
plt.title("Confusion matrix")
plt.xlabel("Predicted value")
plt.ylabel("Actual value")
plt.show()

In [ ]:
test_ids = test['id']
test.drop(columns=['id'], inplace=True)

In [ ]:
probs = pipeline.predict_proba(test)
class_names = le.classes_  # use label encoder mapping
submission = pd.DataFrame(probs, columns=[f"Status_{cls}" for cls in class_names])
submission.insert(0, 'id', test_ids)

In [ ]:
submission.to_csv("submission3.csv", index=False)
print("\n✅ Submission file created successfully!")
print(submission.head())